In [4]:
if __name__=="__main__":
    try:
        from pyspark.sql import SparkSession
    except:
        import findspark
        findspark.init()
        from pyspark.sql import SparkSession
    spark=SparkSession.builder \
    .master("local") \
    .appName("kmeans") \
    .getOrCreate()
    sc=spark.sparkContext

In [1]:
r=sc.parallelize([1,2,3,4])
type(r)

pyspark.rdd.RDD

In [32]:
r1=sc.parallelize(['manzana',1,1.4])
r2=sc.parallelize([i for i in range(10)])

In [33]:
print(r1.collect())
print(r2.collect())

['manzana', 1, 1.4]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [6]:
print(r2.count())
print(r2.mean())
print(r2.variance())
print(r2.take(5))

10
4.5
8.25
[0, 1, 2, 3, 4]


In [7]:
text=sc.textFile('textSpark.txt')
#text.collect()
text.take(1)

['Apache Spark es un framework de computación en clúster open-source. Fue desarrollada originariamente en la Universidad de California, en el AMPLab de Berkeley. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.']

In [ ]:
#### Acciones

In [10]:
def add(x,y):
    return(x+y)

In [11]:
r2.reduce(add)

45

In [34]:
r2.reduce(lambda x,y:x+y)

45

In [7]:
r3=sc.parallelize(range(0,100),3)
result=r3.reduce(lambda x,y:x-y)
result

3696

In [35]:
type(result)

int

In [17]:
r3=sc.parallelize(range(0,5),2)
r3.reduce(lambda x,y:x-y)

4

In [18]:
text.aggregate(0,lambda c,s:c+s.count('a'),lambda n1,n2:n1+n2)

91

In [21]:
number=sc.parallelize(range(0,500),2)
#number.take(10)

In [26]:
#number.saveAsTextFile("C:\tmp\datos\data.txt")

In [ ]:
##### Transformacion

In [30]:
def incremento(x):
    return x+2

In [31]:
r4=r2.map(incremento)
print(r4.collect())
print(r4)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
PythonRDD[45] at collect at <ipython-input-31-1576fa55ef62>:2


In [36]:
def par(x):
    if x%2==0:
        return True
    return False

In [37]:
r5=r2.filter(par)
print(r5.collect())
print(type(r5))

[0, 2, 4, 6, 8]
<class 'pyspark.rdd.PipelinedRDD'>


In [40]:
r6=sc.parallelize(range(0,6))
r7=sc.parallelize(range(5,9))

In [39]:
r8=r6.union(r7)
r8.collect()

[0, 1, 2, 3, 4, 5, 5, 6, 7, 8]

In [41]:
r9=r6.intersection(r7)
r9.collect()

[5]

In [61]:
### SparkSQL

In [43]:
l=[('juan',29),('pedro',19),('jose',32)]
df=spark.createDataFrame(l,['nombre','edad'])
type(df)

pyspark.sql.dataframe.DataFrame

In [44]:
df.show()
type(df)

+------+----+
|nombre|edad|
+------+----+
|  juan|  29|
| pedro|  19|
|  jose|  32|
+------+----+



pyspark.sql.dataframe.DataFrame

In [46]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- edad: long (nullable = true)



In [65]:
if sc.version.startswith("2"):
     csv_plugin = "csv"
     print('version 2')
else:
     csv_plugin = "com.databricks.spark.csv"

version 2


In [91]:
df0 = sqlContext.read.format(csv_plugin).options(header='true', inferSchema='true').load('titanic.csv')
type(df0)

pyspark.sql.dataframe.DataFrame

In [5]:
df1=spark.read.csv('titanic.csv',header=True,inferSchema=True)
print(type(df1))
print(df1.printSchema())
print(df1.count())

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- survived: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- n_siblings_spouses: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- class: string (nullable = true)
 |-- deck: string (nullable = true)
 |-- embark_town: string (nullable = true)
 |-- alone: string (nullable = true)

None
627


In [6]:
df1=df1.drop('survived','deck')

In [7]:
df1.printSchema()
df1.show()
df1.describe().toPandas()

root
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- n_siblings_spouses: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- class: string (nullable = true)
 |-- embark_town: string (nullable = true)
 |-- alone: string (nullable = true)

+------+----+------------------+-----+-------+------+-----------+-----+
|   sex| age|n_siblings_spouses|parch|   fare| class|embark_town|alone|
+------+----+------------------+-----+-------+------+-----------+-----+
|  male|22.0|                 1|    0|   7.25| Third|Southampton|    n|
|female|38.0|                 1|    0|71.2833| First|  Cherbourg|    n|
|female|26.0|                 0|    0|  7.925| Third|Southampton|    y|
|female|35.0|                 1|    0|   53.1| First|Southampton|    n|
|  male|28.0|                 0|    0| 8.4583| Third| Queenstown|    y|
|  male| 2.0|                 3|    1| 21.075| Third|Southampton|    n|
|female|27.0|                 0|  

,summary,sex,age,n_siblings_spouses,parch,fare,class,embark_town,alone
0,count,627,627,627,627,627,627,627,627
1,mean,None,29.631307814992027,0.5454545454545454,0.379585326953748,34.385398564593245,None,None,None
2,stddev,None,12.511817629565812,1.1510895973422302,0.7929992125432801,54.597730499456304,None,None,None
3,min,female,0.75,0,0,0.0,First,Cherbourg,n
4,max,male,80.0,8,5,512.3292,Third,unknown,y


In [95]:
##### Spark ML 

In [8]:
from pyspark.ml.feature import StringIndexer,OneHotEncoderEstimator,VectorAssembler,MinMaxScaler

In [9]:
sex_enc=StringIndexer(inputCol='sex',outputCol='sex_enc',stringOrderType='alphabetAsc')
embark_enc=StringIndexer(inputCol='embark_town',outputCol='embark_enc',stringOrderType='alphabetAsc')
class_enc=StringIndexer(inputCol='class',outputCol='class_enc',stringOrderType='alphabetAsc')
alone_enc=StringIndexer(inputCol='alone',outputCol='alone_enc',stringOrderType='alphabetAsc')

In [10]:
ohe=OneHotEncoderEstimator(inputCols=['sex_enc','embark_enc'],outputCols=['sex_ohe','embark_ohe'])

In [11]:
vec=VectorAssembler(inputCols=['age','n_siblings_spouses','parch','fare','class_enc','alone_enc',
                               'sex_ohe','embark_ohe'],outputCol='features')

In [12]:
sca=MinMaxScaler(inputCol='features',outputCol='features_mm')

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [15]:
clu=KMeans(k=3)
pipe=Pipeline(stages=[sex_enc,embark_enc,class_enc,alone_enc,ohe,vec,sca,clu])

In [16]:
modelo=pipe.fit(df1)

In [17]:
prediccion=modelo.transform(df1)
#prediccion.select('prediction').show(5)

In [18]:
evaluacion=ClusteringEvaluator()
print('Silhoutte', evaluacion.evaluate(prediccion))

Silhoutte 0.8185907502863816
